In [1]:
import h2o
from h2o.automl import H2OAutoML

# Initializing the H2O server
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.2+9-LTS, mixed mode)
  Starting server from C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\User\AppData\Local\Temp\tmpb14jtv0a
  JVM stdout: C:\Users\User\AppData\Local\Temp\tmpb14jtv0a\h2o_User_started_from_python.out
  JVM stderr: C:\Users\User\AppData\Local\Temp\tmpb14jtv0a\h2o_User_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Helsinki
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_User_8xyhjq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.986 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [2]:
# Loading the dataset
df = h2o.import_file('../data/df_prepped.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [3]:
# Filter the rows where 'Year' is 2016
df_test = df[df['Year'] == 2016]

# Sort the filtered data by 'Year'
df_test = df_test.sort('Year')

# Filter the rows where 'Year' is not 2016
df_train = df[df['Year'] != 2016]


In [4]:
# Identifying predictors and target
y = 'Y_maize_major'
x = df.columns
x.remove(y)

In [5]:
# For categorical columns, ensure they are handled as factors
df_train['Countries'] = df_train['Countries'].asfactor()
df_train['Farm'] = df_train['Farm'].asfactor()
df_test['Countries'] = df_test['Countries'].asfactor()
df_test['Farm'] = df_test['Farm'].asfactor()

In [6]:
# Running AutoML for 30 base models (limited here for example purposes)
aml = H2OAutoML(max_models=20, seed=1, nfolds=5)
aml.train(x=x, y=y, training_frame=df_train)

AutoML progress: |
15:15:57.229: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/20
# GBM base models (used / total),8/10
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [7]:
# Viewing the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))  # Printing all rows instead of default (10 rows)

model_id                                                     rmse        mse       mae      rmsle    mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20231216_151557     0.223741  0.05006    0.125271  0.0712712                 0.05006
StackedEnsemble_BestOfFamily_1_AutoML_1_20231216_151557  0.230682  0.0532143  0.128538  0.0727655                 0.0532143
GBM_4_AutoML_1_20231216_151557                           0.230726  0.0532344  0.128641  0.0727762                 0.0532344
GBM_2_AutoML_1_20231216_151557                           0.231498  0.0535913  0.133193  0.0742782                 0.0535913
GBM_3_AutoML_1_20231216_151557                           0.232752  0.0541736  0.131429  0.0737029                 0.0541736
GBM_5_AutoML_1_20231216_151557                           0.234252  0.0548739  0.136408  0.0760232                 0.0548739
GBM_grid_1_AutoML_1_20231216_151557_model_5              0.23895   0.0570971  0.131718  0.0755769                 0.0570971
GBM_grid_1

In [13]:
aml.leader

key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/20
# GBM base models (used / total),8/10
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/7
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [12]:
# Predicting using the leader model on the test set
preds = aml.leader.predict(df_test)

# Combine the actual and predicted values
df_test_actual = df_test['Y_maize_major'].cbind(preds)

# Calculate performance on the test set
perf_test = aml.leader.model_performance(df_test_actual)

# Print MSE, RMSE, and MAE for the test set
print("Test Set Metrics:")
print("MSE:", perf_test.mse())
print("RMSE:", perf_test.rmse())
print("MAE:", perf_test.mae())


stackedensemble prediction progress: |

███████████████████████████████████████████| (done) 100%
Test Set Metrics:
MSE: 3.6969022532755957
RMSE: 1.9227330166394907
MAE: 1.1167138352259294


In [10]:
#saving the model
h2o.save_model(model=aml.leader, path="./my_h2o_model", force=True)


'E:\\AUTOML\\AutoML_Project2\\notebooks\\my_h2o_model\\StackedEnsemble_AllModels_1_AutoML_1_20231216_151557'